In [37]:
import pickle
import random
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfTransformer

In [7]:
## read in the training data
with open("/home/nyao/Code/Data/sentence_score_conso10.pk", 'rb') as f:
    data=pickle.load(f)

In [9]:
data.head(5)

,Website,Sentence,Data Retention,Data Security,Do Not Track,First Party Collection/Use,International and Specific Audiences,Not_used,Policy Change,Third Party Sharing/Collection,"User Access, Edit and Deletion",User Choice/Control
0,playstation.com,"Privacy Policy Last Revised: April, 2011",NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,playstation.com,"Sony Computer Entertainment America LLC (""SCEA...",NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,playstation.com,This privacy policy is intended to provide you...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,playstation.com,"If you have any questions, complaints or comme...",NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,playstation.com,This Privacy Statement and the certification s...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [36]:
datasecurity=data[["Sentence", "Data Security"]]
datasecurity.loc[pd.isnull(datasecurity["Data Security"])]
datasecurity.iloc[:423].shape

(423, 2)

In [14]:
a=[1,2,3,4]
random.shuffle(a)
print(a)

[1, 3, 4, 2]


In [66]:

def getXY(scoretable, category, trainportion):
    # trainportion means the percentage of data to be used as training, the rest are used as validation
    # choose a category, get the data from the data file
    categoryFrame=scoretable[["Sentence", category]]
    nonmissing=categoryFrame.dropna()
    nonmissing=nonmissing.copy()
    nonmissing.loc[:,category]=1
    missingdata=categoryFrame.loc[pd.isnull(categoryFrame[category])]
    missingdata=missingdata.copy()
    missingdata.loc[:,category]=0

    ## equal portion sampling, sample from both positive and negative group
    shufflednonmissing=nonmissing.sample(frac=1)   # shuffle the data 
    shuffledmissing=missingdata.sample(frac=1)

    nonmissingNumber=shufflednonmissing.shape[0]
    missingNumber=shuffledmissing.shape[0]

    trainPos=shufflednonmissing.iloc[0:int(nonmissingNumber*trainportion)]
    valPos=shufflednonmissing.iloc[int(nonmissingNumber*trainportion)+1::]

    trainNeg=shuffledmissing.iloc[0:int(missingNumber*trainportion)]
    valNeg=shuffledmissing.iloc[int(missingNumber*trainportion)+1::]

    trainData=pd.concat([trainPos, trainNeg], axis=0)
    valData=pd.concat([valPos, valNeg], axis=0)

    trainData=trainData.sample(frac=1)
    valData=valData.sample(frac=1)

    trainTexts=trainData["Sentence"].tolist()
    trainLabels=trainData[category].tolist()

    valTexts=valData["Sentence"].tolist()
    valLabels=valData[category].tolist()

    return trainTexts, valTexts, trainLabels, valLabels

def modelAndValidate(datatable,category, vectorizer, classifier, trainportion):
    results={}

    # generate the data
    traintexts, valTexts,trainlabels, valLabels =getXY(datatable, category, trainportion)
    
    # use the vectorizer to generate features, for both training and validation data
    vectorizer.fit(traintexts)
    trainCounts=vectorizer.transform(traintexts)
    valCounts=vectorizer.transform(valTexts)
    
    # fit the model onto the data
    classifier.fit(trainCounts, trainlabels)
    
    # predict the samples in the validation
    predictions=classifier.predict(valCounts)
    f1score=f1_score(valLabels, predictions, average="weighted")
    
    
    # return the results
    results={"model":classifier, "validation_labels":valLabels, "predictions":predictions, "f1_score":f1score}
    print("Finished modeling for category {}".format(category))
    print("F1 score : ", f1score)
    
    return results
    

In [67]:
## initialize a vectorizer
vectorizerFinal=TfidfVectorizer(max_df=0.2, stop_words="english") 
## initialize a classifer

classifierFinal=RandomForestClassifier(n_estimators=500)

category="Data Security"
results = modelAndValidate(data, category, vectorizerFinal, classifierFinal, 0.8)
print(results["f1_score"])

Finished modeling for category Data Security
F1 score :  0.951169555546
0.951169555546
